In [2]:
import numpy as np
import pickle
import operator
from keras_transformer import get_model, decode
# main_path = '/content/drive/My Drive/Colab Notebooks/'    #Google Colab FilePath
main_path = './'
path = main_path + 'middle_data/'
path = 'middle_data/'
with open(path + 'encode_input.pkl', 'rb') as f:
    encode_input = pickle.load(f)
with open(path + 'decode_input.pkl', 'rb') as f:
    decode_input = pickle.load(f)
with open(path + 'decode_output.pkl', 'rb') as f:
    decode_output = pickle.load(f)
with open(path + 'source_token_dict.pkl', 'rb') as f:
    source_token_dict = pickle.load(f)
with open(path + 'target_token_dict.pkl', 'rb') as f:
    target_token_dict = pickle.load(f)
with open(path + 'source_tokens.pkl', 'rb') as f:
    source_tokens = pickle.load(f)
print('Done')

Done


In [3]:
print(len(source_token_dict))
print(len(target_token_dict))
print(len(encode_input))
# 构建模型
model = get_model(
    token_num=max(len(source_token_dict), len(target_token_dict)),
    embed_dim=64,
    encoder_num=2,
    decoder_num=2,
    head_num=4,
    hidden_dim=256,
    dropout_rate=0.05,
    use_same_embed=False,  # 不同语言需要使用不同的词嵌入
)
model.compile('adam', 'sparse_categorical_crossentropy')
# model.summary()
print('Done')

10565
7307
20403
Done


In [4]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print("🔍 TensorFlow 版本:", tf.__version__)
print("🧠 是否有 GPU:", tf.config.list_physical_devices('GPU'))

print("\n📦 本地设备信息:")
for device in device_lib.list_local_devices():
    print(device)


🔍 TensorFlow 版本: 2.19.0
🧠 是否有 GPU: []

📦 本地设备信息:
name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10085201896275080810
xla_global_id: -1



In [ ]:
import os
import sentencepiece as spm
import numpy as np
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Dropout
from keras.layers import LayerNormalization, MultiHeadAttention
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# === 1. 数据加载和 SentencePiece 训练 ===
data_path = 'cmn.txt'  # 每行格式: 英文 \t 中文
with open(data_path, encoding='utf-8') as f:
    lines = [l.strip() for l in f if '\t' in l]
pairs = [l.split('\t') for l in lines]
en_texts, zh_texts = zip(*pairs)

# 生成混合训练文件
with open('combined.txt', 'w', encoding='utf-8') as f:
    for en, zh in zip(en_texts, zh_texts):
        f.write(f"<to_en> {zh}\n")
        f.write(f"<to_zh> {en}\n")

# 训练共享词表
if not os.path.exists('spm.model'):
    spm.SentencePieceTrainer.train(
        input='combined.txt', model_prefix='spm',
        vocab_size=16000, model_type='bpe', character_coverage=1.0,
        control_symbols=['<PAD>', '<START>', '<END>', '<to_en>', '<to_zh>']
    )

# === 2. 加载 SentencePiece 模型 ===
sp = spm.SentencePieceProcessor(model_file='spm.model')
pad_id = sp.piece_to_id('<PAD>')
start_id = sp.piece_to_id('<START>')
end_id = sp.piece_to_id('<END>')
to_en_id = sp.piece_to_id('<to_en>')
to_zh_id = sp.piece_to_id('<to_zh>')

# === 3. 编码样本对 ===
def encode_pair(en, zh):
    zh2en_enc = [to_en_id] + sp.encode(zh, out_type=int)
    zh2en_dec_in = [start_id] + sp.encode(en, out_type=int)
    zh2en_dec_out = sp.encode(en, out_type=int) + [end_id]

    en2zh_enc = [to_zh_id] + sp.encode(en, out_type=int)
    en2zh_dec_in = [start_id] + sp.encode(zh, out_type=int)
    en2zh_dec_out = sp.encode(zh, out_type=int) + [end_id]

    return [
        zh2en_enc, zh2en_dec_in, zh2en_dec_out,
        en2zh_enc, en2zh_dec_in, en2zh_dec_out
    ]

all_enc_in, all_dec_in, all_dec_out = [], [], []
for en, zh in zip(en_texts, zh_texts):
    a, b, c, d, e, f = encode_pair(en, zh)
    all_enc_in.extend([a, d])
    all_dec_in.extend([b, e])
    all_dec_out.extend([c, f])

MAX_LEN = max(map(len, all_enc_in + all_dec_in + all_dec_out))
print("MAX_LEN =", MAX_LEN)

X_enc = pad_sequences(all_enc_in, maxlen=MAX_LEN, padding='post', value=pad_id)
X_dec = pad_sequences(all_dec_in, maxlen=MAX_LEN, padding='post', value=pad_id)
Y_out = pad_sequences(all_dec_out, maxlen=MAX_LEN, padding='post', value=pad_id)
Y_out = np.expand_dims(Y_out, -1)

# === 4. 构建简化 Transformer 模型 ===
def transformer_block(inputs, heads, d_ff, dropout):
    attn_output = MultiHeadAttention(num_heads=heads, key_dim=64)(inputs, inputs)
    out1 = LayerNormalization()(inputs + attn_output)
    ff = Dense(d_ff, activation='relu')(out1)
    ff = Dense(inputs.shape[-1])(ff)
    return LayerNormalization()(out1 + Dropout(dropout)(ff))

def build_model(vocab_size, max_len):
    inp_enc = Input(shape=(max_len,))
    inp_dec = Input(shape=(max_len,))

    embed = Embedding(vocab_size, 256, mask_zero=True)
    enc = embed(inp_enc)
    dec = embed(inp_dec)

    enc = transformer_block(enc, heads=4, d_ff=512, dropout=0.1)
    dec = transformer_block(dec, heads=4, d_ff=512, dropout=0.1)

    out = Dense(vocab_size, activation='softmax')(dec)
    model = Model([inp_enc, inp_dec], out)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model(vocab_size=sp.get_piece_size(), max_len=MAX_LEN)
model.summary()

# === 5. 模型训练 ===
checkpoint = ModelCheckpoint('best_model.weights.h5', save_best_only=True, save_weights_only=True, monitor='loss', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=2, verbose=1)

model.fit(
    [X_enc, X_dec], Y_out,
    epochs=20,
    batch_size=64,
    callbacks=[checkpoint, reduce_lr],
    verbose=1  # 显示每个 batch 的训练过程
)


In [ ]:
#加载模型
model.load_weights('models/W--010-0.5277-.weights.h5')
target_token_dict_inv = {v: k for k, v in target_token_dict.items()}
print('Done')

In [ ]:
from keras.preprocessing import sequence
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import jieba
import requests

import re

# 判断是否包含中文字符
def contains_chinese(text):
    return bool(re.search(r'[\u4e00-\u9fff]', text))

# 获取输入序列并转换为编码（支持中英互译）
def get_input(seq, is_chinese):
    if is_chinese:
        seq = ' '.join(jieba.lcut(seq, cut_all=False))
        seq = seq.split(' ')
        token_dict = source_token_dict
    else:
        seq = seq.strip().split(' ')
        token_dict = target_token_dict

    seq = ['<START>'] + seq + ['<END>']
    seq = seq + ['<PAD>'] * (34 - len(seq))
    
    for x in seq:
        if x not in token_dict:
            return False, []
    seq_ids = [token_dict[x] for x in seq]
    return True, seq_ids

# 翻译并输出结果
def get_ans(seq_ids, is_chinese):
    decoded = decode(
        model,
        [seq_ids],
        start_token=(target_token_dict if is_chinese else source_token_dict)['<START>'],
        end_token=(target_token_dict if is_chinese else source_token_dict)['<END>'],
        pad_token=(target_token_dict if is_chinese else source_token_dict)['<PAD>'],
    )
    token_dict_inv = target_token_dict_inv if is_chinese else {v: k for k, v in source_token_dict.items()}
    print(' '.join(map(lambda x: token_dict_inv[x], decoded[0][1:-1])))

# 循环交互
while True:
    seq = input("请输入中英文句子 (输入 'x' 退出): ")
    if seq.strip().lower() == 'x':
        break
    is_chinese = contains_chinese(seq)
    flag, seq_ids = get_input(seq, is_chinese)
    if flag:
        get_ans(seq_ids, is_chinese)
    else:
        print('听不懂呢。')